## SPHINCS+

#### Pedro Gonçalves (A82313) & Roberto Cachada (A81012)

Os algoritmos/notação implementados/utilizados neste *notebook* são baseados no [documento](https://sphincs.org/data/sphincs+-round2-specification.pdf) da segunda ronda da candidatura ao concurso da NIST.

#### Parâmetros SPHINCS+

A assinatura SPHINCS+ é um esquema de assinatura *stateless* baseado em hashes. Esta possui 6 parâmetros, como se pode verificar no capítulo 7.1 do documento de candidatura, sendo eles:
- n -> Parâmetro de segurança
- w -> Parâmetro de Winternitz
- h -> Altura da Hypertree
- d -> Número de camadas da Hypertree
- k -> Número de árvores na FORS
- t -> Número de folhas de uma árvore FORS

In [1]:
#Parámetros SPHINCS+ 128 Simple

import math
import hashlib
import random
import os

RANDOMIZE = True #Parâmetro que indica se o utilizador quer atribuir aleatoriedade às assinaturas

n = 16
w = 16
h = 64
d = 8
k = 10
a = 15
t = 2 ** a #a = log2(t) <=> t 2^a

len1 = math.ceil(8 * n / math.log(w, 2))
len2 = math.floor(math.log(len1 * (w - 1), 2) / math.log(w, 2)) + 1
l= len1 + len2
hPrime = h // d

#### ADRS

Uma *address* (ADRS), é um valor de 32 *bytes* que segue uma estrutura pré-definida, que possui um conjunto de *sets* que permitem a sua manipulação. Existem 5 tipos de  *addresses*, como é explicado no capítulo 2.7.3 do [documento](https://sphincs.org/data/sphincs+-round2-specification.pdf) em que esta implementação se baseia.

In [2]:
class ADRS:
    #Tipos ADRS definidos no capítulo 2.7.3
    WOTS_HASH = 0
    WOTS_PK = 1
    TREE = 2
    FORS_TREE = 3
    FORS_ROOTS = 4

    def __init__(self):
        self.layer = 0
        self.tree_address = 0
        self.type = 0
        #Estas últimas words variam conforme o tipo de ADRS
        self.word_1 = 0
        self.word_2 = 0
        self.word_3 = 0

    def copy(self):
        adrs = ADRS()
        adrs.layer = self.layer
        adrs.tree_address = self.tree_address
        adrs.type = self.type
        adrs.word_1 = self.word_1
        adrs.word_2 = self.word_2
        adrs.word_3 = self.word_3
        return adrs

    #Produz um array de bytes
    #Cada word possui o tamanho de 32 bits, excepto a tree_address, que possui o tamanho de 3 words
    def to_bin(self):
        res = self.layer.to_bytes(4, byteorder='big')
        res += self.tree_address.to_bytes(12, byteorder='big')
        res += self.type.to_bytes(4, byteorder='big')
        res += self.word_1.to_bytes(4, byteorder='big')
        res += self.word_2.to_bytes(4, byteorder='big')
        res = self.word_3.to_bytes(4, byteorder='big')

        return res

#---------------------
# Setts & Getters
    
    def set_type(self, val):
        self.type = val
        self.word_2 = 0
        self.word_3 = 0
        self.word_1 = 0

    def set_layer_address(self, val):
        self.layer = val

    def set_tree_address(self, val):
        self.tree_address = val

    def set_key_pair_address(self, val):
        self.word_1 = val

    def set_chain_address(self, val):
        self.word_2 = val

    def set_hash_address(self, val):
        self.word_3 = val

    def set_tree_height(self, val):
        self.word_2 = val

    def set_tree_index(self, val):
        self.word_3 = val
            
    def get_tree_height(self):
        return self.word_2

    def get_key_pair_address(self):
        return self.word_1

    def get_tree_index(self):
        return self.word_3
    
    def reset_words(self):
        self.word_1 = 0
        self.word_2 = 0
        self.word_3 = 0

#### Funções auxiliares
Neste capítulo serão apresentadas algumas funções auxiliares, que serão úteis no desenvolver dos algoritmos do SPHINCS+.

###### 1) *base_w*
Uma string, x, pode ser considerada uma *string* de números base w, i.e. inteiros que pertencem ao set {0,...,w-1}. Assim sendo, foi definida, a função ***base_w*** para fazer essa conversão.

###### 2) *Tweakable Hash Functions*
Este tipo de funções recebem uma uma *seed*, uma mensagem e informação em forma de uma **ADRS**. Isto permite que fazer com que a chamada da função de *hash* para cada *key-pair* das estruturas virtuais do SPHINCS+ sejam independentes para cada uma.

In [3]:
# Conversão de uma string de bytes para uma string de números base w (Capítulo 2.5) 
# Input: len_X-byte string X, int w, output length out_len
# Output: out_len int array basew
def base_w(x, w, out_len):
    vin = 0
    vout = 0
    total = 0
    bits = 0
    basew = []

    for consumed in range(0, out_len):
        if bits == 0:
            total = x[vin]
            vin += 1
            bits += 8
        bits -= math.floor(math.log(w, 2))
        basew.append((total >> bits) % w)
        vout += 1

    return basew

#---------------------
# Tweakable Hash Functions (Capítulo 2.7)
def hash(seed, adrs: ADRS, value, digest_size):
    m = hashlib.sha256()
    m.update(seed)
    m.update(adrs.to_bin())
    m.update(value)
    hashed = m.digest()[:digest_size]
    return hashed

# Função que serve para comprimir a mensagem antes da assinatura
def hash_msg(r, public_seed, public_root, value, digest_size):
    m = hashlib.sha256()
    m.update(r)
    m.update(public_seed)
    m.update(public_root)
    m.update(value)
    hashed = m.digest()[:digest_size]
    i = 0

    while len(hashed) < digest_size:
        i += 1
        m = hashlib.sha256()
        m.update(r)
        m.update(public_seed)
        m.update(public_root)
        m.update(value)
        m.update(bytes([i]))
        hashed += m.digest()[:digest_size - len(hashed)]

    return hashed

# Função pseudorandom que serve para pseudorandom key generation
def prf(secret_seed, adrs, digest_size):
    random.seed(int.from_bytes(secret_seed + adrs.to_bin(), "big"))
    return random.randint(0, 256 ** digest_size - 1).to_bytes(digest_size, byteorder='big')

# Função pseudorandom para gerar "randomness" para a compressão das mensagens
def prf_msg(secret_seed, opt, m, digest_size):
    random.seed(int.from_bytes(secret_seed + opt + hash_msg(b'0', b'0', b'0', m, digest_size * 2), "big"))
    return random.randint(0, 256 ** digest_size - 1).to_bytes(digest_size, byteorder='big')

#---------------------
    
def sig_wots_from_sig_xmss(sig):
    return sig[0:l]

def auth_from_sig_xmss(sig):
    return sig[l:]

def sigs_xmss_from_sig_ht(sig):
    sigs = []
    for i in range(0, d):
        sigs.append(sig[i * (hPrime + l):(i + 1) * (hPrime + l)])

    return sigs

def auths_from_sig_fors(sig):
    sigs = []
    for i in range(0, k):
        sigs.append([])
        sigs[i].append(sig[(a + 1) * i])
        sigs[i].append(sig[((a + 1) * i + 1):((a + 1) * (i + 1))])

    return sigs

#### WOTS+

WOTS+ é um esquema de *One Time Signature*, o que significa que nenhuma chave privada não deve ser usada mais do que uma única mensagem, pois se uma mesma chave privada assinar duas mensagens diferentes põe em causa a segurança do esquema. O WOTS+ possui como parâmetros o n, w, len, len1 e len2.
As implementações apresentadas em baixo seguem os algoritmos presentes no capítulo 3 da documentação do SPHINCS+.

In [4]:
# Chaining function used in WOTS+ (Capítulo 3.2)
# Input: Input string X, start index i, number of steps s, public seed PK.seed, address ADRS
# Output: value of F iterated s times on X
def chain(x, i, s, public_seed, adrs: ADRS):
    if s == 0:
        return bytes(x)
    if (i + s) > (w- 1):
        return -1
    tmp = chain(x, i, s - 1, public_seed, adrs)
    adrs.set_hash_address(i + s - 1)
    tmp = hash(public_seed, adrs, tmp, n)

    return tmp

# Generating a WOTS+ private key (Capítulo 3.3)
# Input: secret seed SK.seed, address ADRS
# Output: WOTS+ private key sk
def wots_sk_gen(secret_seed, adrs: ADRS):
    sk = []
    for i in range(0, l):
        adrs.set_chain_address(i)
        adrs.set_hash_address(0)
        sk.append(prf(secret_seed, adrs.copy(), n))
    return sk

# Generating a WOTS+ public key (Capítulo 3.4)
# Input: secret seed SK.seed, address ADRS, public seed PK.seed
# Output: WOTS+ public key pk
def wots_pk_gen(secret_seed, public_seed, adrs: ADRS):
    wots_pk_adrs = adrs.copy()
    tmp = bytes()
    
    for i in range(0, l):
        adrs.set_chain_address(i)
        adrs.set_hash_address(0)
        sk = prf(secret_seed, adrs.copy(), n)
        tmp += bytes(chain(sk, 0, w- 1, public_seed, adrs.copy()))

    wots_pk_adrs.set_type(ADRS.WOTS_PK)
    wots_pk_adrs.set_key_pair_address(adrs.get_key_pair_address())
    pk = hash(public_seed, wots_pk_adrs, tmp, n)
    return pk

# Generating a WOTS+ signature on a message M (Capítulo 3.5)
# Input: Message M, secret seed SK.seed, public seed PK.seed, address ADRS
# Output: WOTS+ signature sig
def wots_sign(m, secret_seed, public_seed, adrs):
    csum = 0
    msg = base_w(m, w, len1)
    for i in range(0, len1):
        csum += w- 1 - msg[i]
        
    padding = (len2 * math.floor(math.log(w, 2))) % 8 if (len2 * math.floor(math.log(w, 2))) % 8 != 0 else 8
    csum = csum << (8 - padding)
    csumb = csum.to_bytes(math.ceil((len2 * math.floor(math.log(w, 2))) / 8), byteorder='big')
    csumw = base_w(csumb, w, len2)
    msg += csumw
    sig = []

    for i in range(0, l):
        adrs.set_chain_address(i)
        adrs.set_hash_address(0)
        sk = prf(secret_seed, adrs.copy(), n)
        sig += [chain(sk, 0, msg[i], public_seed, adrs.copy())]

    return sig

# Computing a WOTS+ public key from a message and its signature. (Capítulo 3.6)
# Input: Message M, WOTS+ signature sig, address ADRS, public seed PK.seed
# Output: WOTS+ public key pk_sig derived from sig
def wots_pk_from_sig(sig, m, public_seed, adrs: ADRS):
    csum = 0
    wots_pk_adrs = adrs.copy()
    msg = base_w(m, w, len1)
    
    for i in range(0, len1):
        csum += w- 1 - msg[i]

    padding = (len2 * math.floor(math.log(w, 2))) % 8 if (len2 * math.floor(math.log(w, 2))) % 8 != 0 else 8
    csum = csum << (8 - padding)
    csumb = csum.to_bytes(math.ceil((len2 * math.floor(math.log(w, 2))) / 8), byteorder='big')
    csumw = base_w(csumb, w, len2)
    msg += csumw

    tmp = bytes()
    for i in range(0, l):
        adrs.set_chain_address(i)
        tmp += chain(sig[i], msg[i], w- 1 - msg[i], public_seed, adrs.copy())

    wots_pk_adrs.set_type(ADRS.WOTS_PK)
    wots_pk_adrs.set_key_pair_address(adrs.get_key_pair_address())
    pk_sig = hash(public_seed, wots_pk_adrs, tmp, n)
    return pk_sig

#### Hypertree

Neste capítulo, serão implementadas as funções que fazem a combinação de WOTS+ com uma árvore binária de hashes, que cria uma versão de comprimento de input fixo de um *eXtended Merkle Signature Scheme (XMSS)*. Depois são também implementadas as funções para a conversão de **XMSS** para uma *hypertree*, que pode ser vista como uma árvore de várias camadas de árvores **XMSS**. As camadas intermédias/superiores são utilizadas para assinar as chaves públicas.

In [5]:
# XMSS

# The TreeHash algorithm (Capítulo 4.1.3)
# Input: Secret seed SK.seed, start index s, target node height z, public seed PK.seed, address ADRS
# Output: n-byte root node - top node on Stack
def treehash(secret_seed, s, z, public_seed, adrs: ADRS):
    if s % (1 << z) != 0:
        return -1

    stack = []

    for i in range(0, 2 ** z):
        adrs.set_type(ADRS.WOTS_HASH)
        adrs.set_key_pair_address(s + i)
        node = wots_pk_gen(secret_seed, public_seed, adrs.copy())
        adrs.set_type(ADRS.TREE)
        adrs.set_tree_height(1)
        adrs.set_tree_index(s + i)

        if len(stack) > 0:
            while stack[len(stack) - 1]['height'] == adrs.get_tree_height():
                adrs.set_tree_index((adrs.get_tree_index() - 1) // 2)
                node = hash(public_seed, adrs.copy(), stack.pop()['node'] + node, n)
                adrs.set_tree_height(adrs.get_tree_height() + 1)

                if len(stack) <= 0:
                    break

        stack.append({'node': node, 'height': adrs.get_tree_height()})

    return stack.pop()['node']

# Generating an XMSS public key (Capítulo 4.1.4)
# Input: Secret seed SK.seed, public seed PK.seed, address ADRS
# Output: XMSS public key PK
def xmss_pk_gen(secret_seed, public_key, adrs: ADRS):
    pk = treehash(secret_seed, 0, hPrime, public_key, adrs.copy())
    return pk

# Generating an XMSS signature (Capítulo 4.1.6)
# Input: n-byte message M, secret seed SK.seed, index idx, public seed PK.seed, address ADRS
# Output: XMSS signature SIG_XMSS = (sig || AUTH)
def xmss_sign(m, secret_seed, idx, public_seed, adrs):
    auth = []
    for j in range(0, hPrime):
        ki = math.floor(idx // 2 ** j)
        if ki % 2 == 1:
            ki -= 1
        else:
            ki += 1

        auth += [treehash(secret_seed, ki * 2 ** j, j, public_seed, adrs.copy())]

    adrs.set_type(ADRS.WOTS_HASH)
    adrs.set_key_pair_address(idx)
    sig = wots_sign(m, secret_seed, public_seed, adrs.copy())
    sig_xmss = sig + auth
    return sig_xmss

# Computing an XMSS public key from an XMSS signature (Capítulo 4.1.7)
# Input: index idx, XMSS signature SIG_XMSS = (sig || AUTH), n-byte message M, public seed PK.seed, address ADRS
# Output: n-byte root value node[0]
def xmss_pk_from_sig(idx, sig_xmss, m, public_seed, adrs):
    adrs.set_type(ADRS.WOTS_HASH)
    adrs.set_key_pair_address(idx)
    sig = sig_wots_from_sig_xmss(sig_xmss)
    auth = auth_from_sig_xmss(sig_xmss)
    node_0 = wots_pk_from_sig(sig, m, public_seed, adrs.copy())
    node_1 = 0
    adrs.set_type(ADRS.TREE)
    adrs.set_tree_index(idx)

    for i in range(0, hPrime):
        adrs.set_tree_height(i + 1)
        if math.floor(idx / 2 ** i) % 2 == 0:
            adrs.set_tree_index(adrs.get_tree_index() // 2)
            node_1 = hash(public_seed, adrs.copy(), node_0 + auth[i], n)
        else:
            adrs.set_tree_index((adrs.get_tree_index() - 1) // 2)
            node_1 = hash(public_seed, adrs.copy(), auth[i] + node_0, n)

        node_0 = node_1

    return node_0

#---------------------
# HT

# Generating an HT public key (Capítulo 4.2.2)
# Input: Private seed SK.seed, public seed PK.seed
# Output: HT public key PK_HT
def ht_pk_gen( secret_seed, public_seed):
    adrs = ADRS()
    adrs.set_layer_address(d - 1)
    adrs.set_tree_address(0)
    root = xmss_pk_gen(secret_seed, public_seed, adrs.copy())
    return root

# Generating an HT signature (Capítulo 4.2.4)
# Input: Message M, private seed SK.seed, public seed PK.seed, tree index idx_tree, leaf index idx_leaf
# Output: HT signature SIG_HT
def ht_sign(m, secret_seed, public_seed, idx_tree, idx_leaf):
    adrs = ADRS()
    adrs.set_layer_address(0)
    adrs.set_tree_address(idx_tree)
    sig_tmp = xmss_sign(m, secret_seed, idx_leaf, public_seed, adrs.copy())
    sig_ht = sig_tmp
    root = xmss_pk_from_sig(idx_leaf, sig_tmp, m, public_seed, adrs.copy())

    for j in range(1, d):
        idx_leaf = idx_tree % 2 ** hPrime
        idx_tree = idx_tree >> hPrime

        adrs.set_layer_address(j)
        adrs.set_tree_address(idx_tree)

        sig_tmp = xmss_sign(root, secret_seed, idx_leaf, public_seed, adrs.copy())
        sig_ht = sig_ht + sig_tmp

        if j < d - 1:
            root = xmss_pk_from_sig(idx_leaf, sig_tmp, root, public_seed, adrs.copy())

    return sig_ht

# Verifying a HT signature SIGHT on a message M using a HT public key PKHT (Capítulo 4.2.5)
# Input: Message M, signature SIG_HT, public seed PK.seed, tree index idx_tree, leaf index idx_leaf, HT public key PK_HT
# Output: Boolean
def ht_verify(m, sig_ht, public_seed, idx_tree, idx_leaf, public_key_ht):
    adrs = ADRS()
    sigs_xmss = sigs_xmss_from_sig_ht(sig_ht)
    sig_tmp = sigs_xmss[0]
    adrs.set_layer_address(0)
    adrs.set_tree_address(idx_tree)
    node = xmss_pk_from_sig(idx_leaf, sig_tmp, m, public_seed, adrs)

    for j in range(1, d):
        idx_leaf = idx_tree % 2 ** hPrime
        idx_tree = idx_tree >> hPrime

        sig_tmp = sigs_xmss[j]

        adrs.set_layer_address(j)
        adrs.set_tree_address(idx_tree)

        node = xmss_pk_from_sig(idx_leaf, sig_tmp, node, public_seed, adrs)

    if node == public_key_ht:
        return True
    else:
        return False

#### FORS

Tendo em conta que não é usada a *Hypertree* para assinar mensagens, são usadas as chaves públicas das instâncias **FORS** (Forest of Random Subsets) para fazer essa assinatura.
Os parâmetros da **FORS** são o n, k e o t que representa o número de elementos por chave privada, número de folhas por *hash tree* e *upper bound* dos valores de index.

A assinatura SPHINCS+ utiliza verificação explícita para FORS, utilizando simplesmente um método para cálculo de uma chave pública candidata de uma assinatura.

In [6]:
# Computing a FORS private key value (Capítulo 5.2)
# Input: secret seed SK.seed, address ADRS, secret key index idx = it+j
# Output: FORS private key sk
def fors_sk_gen( secret_seed, adrs: ADRS, idx):
    adrs.set_tree_height(0)
    adrs.set_tree_index(idx)
    sk = prf(secret_seed, adrs.copy(), n)

    return sk

# The fors_treehash algorithm (Capítulo 5.3)
# Input: Secret seed SK.seed, start index s, target node height z, public seed PK.seed, address ADRS
# Output: n-byte root node - top node on Stack
def fors_treehash(secret_seed, s, z, public_seed, adrs):
    if s % (1 << z) != 0:
        return -1

    stack = []

    for i in range(0, 2 ** z):
        adrs.set_tree_height(0)
        adrs.set_tree_index(s + i)
        sk = prf(secret_seed, adrs.copy(), n)
        node = hash(public_seed, adrs.copy(), sk, n)

        adrs.set_tree_height(1)
        adrs.set_tree_index(s + i)
        if len(stack) > 0:
            while stack[len(stack) - 1]['height'] == adrs.get_tree_height():
                adrs.set_tree_index((adrs.get_tree_index() - 1) // 2)
                node = hash(public_seed, adrs.copy(), stack.pop()['node'] + node, n)

                adrs.set_tree_height(adrs.get_tree_height() + 1)

                if len(stack) <= 0:
                    break
        stack.append({'node': node, 'height': adrs.get_tree_height()})

    return stack.pop()['node']

# Generate a FORS public key (Capítulo 5.4)
# Input: Secret seed SK.seed, public seed PK.seed, address ADRS
# Output: FORS public key PK
def fors_pk_gen(secret_seed, public_seed, adrs: ADRS):
    fors_pk_adrs = adrs.copy()
    root = bytes()

    for i in range(0, k):
        root += fors_treehash(secret_seed, i * t, a, public_seed, adrs)

    fors_pk_adrs.set_type(ADRS.FORS_ROOTS)
    fors_pk_adrs.set_key_pair_address(adrs.get_key_pair_address())
    pk = hash(public_seed, fors_pk_adrs, root, n)
    return pk

# Generating a FORS signature on string M (Capítulo 5.5)
# Input: Bit string M, secret seed SK.seed, address ADRS, public seed PK.seed
# Output: FORS signature SIG_FORS
def fors_sign(m, secret_seed, public_seed, adrs):
    m_int = int.from_bytes(m, 'big')
    sig_fors = []
    
    for i in range(0, k):
        idx = (m_int >> (k - 1 - i) * a) % t
        adrs.set_tree_height(0)
        adrs.set_tree_index(i * t + idx)
        sig_fors += [prf(secret_seed, adrs.copy(), n)]

        auth = []
        for j in range(0, a):
            s = math.floor(idx // 2 ** j)
            if s % 2 == 1:
                s -= 1
            else:
                s += 1

            auth += [fors_treehash(secret_seed, i * t + s * 2 ** j, j, public_seed, adrs.copy())]

        sig_fors += auth

    return sig_fors

# Compute a FORS public key from a FORS signature (Capítulo 5.6)
# Input: FORS signature SIG_FORS, (k lg t)-bit string M, public seed PK.seed, address ADRS
# Output: FORS public key
def fors_pk_from_sig(sig_fors, m, public_seed, adrs: ADRS):
    m_int = int.from_bytes(m, 'big')
    sigs = auths_from_sig_fors(sig_fors)
    root = bytes()

    for i in range(0, k):
        idx = (m_int >> (k - 1 - i) * a) % t
        sk = sigs[i][0]
        adrs.set_tree_height(0)
        adrs.set_tree_index(i * t + idx)
        node_0 = hash(public_seed, adrs.copy(), sk, n)
        node_1 = 0
        auth = sigs[i][1]
        adrs.set_tree_index(i * t + idx)

        for j in range(0, a):
            adrs.set_tree_height(j + 1)

            if math.floor(idx / 2 ** j) % 2 == 0:
                adrs.set_tree_index(adrs.get_tree_index() // 2)
                node_1 = hash(public_seed, adrs.copy(), node_0 + auth[j], n)
            else:
                adrs.set_tree_index((adrs.get_tree_index() - 1) // 2)
                node_1 = hash(public_seed, adrs.copy(), auth[j] + node_0, n)

            node_0 = node_1

        root += node_0

    fors_pk_adrs = adrs.copy()
    fors_pk_adrs.set_type(ADRS.FORS_ROOTS)
    fors_pk_adrs.set_key_pair_address(adrs.get_key_pair_address())
    pk = hash(public_seed, fors_pk_adrs, root, n)
    
    return pk

##### SPHINCS+

A explicação dos vários métodos implementados em baixo, encontra-se no capítulo 7 do documento de referência para esta implementação do SPHINCS+.

In [7]:
# Generate a SPHINCS+ key pair (Capítulo 6.2)
# Input: (none)
# Output: SPHINCS+ key pair (SK,PK)
def spx_keygen():
    secret_seed = os.urandom(n)
    secret_prf = os.urandom(n)
    public_seed = os.urandom(n)

    public_root = ht_pk_gen(secret_seed, public_seed)

    return [secret_seed, secret_prf, public_seed, public_root], [public_seed, public_root]

# Generating a SPHINCS+ signature (Capítulo 6.4)
# Input: Message M, private key SK = (SK.seed, SK.prf, PK.seed, PK.root)
# Output: SPHINCS+ signature SIG
def spx_sign(m, secret_key):
    adrs = ADRS()
    secret_seed = secret_key[0]
    secret_prf = secret_key[1]
    public_seed = secret_key[2]
    public_root = secret_key[3]
    opt = bytes(n)
    
    if RANDOMIZE:
        opt = os.urandom(n)
    r = prf_msg(secret_prf, opt, m, n)
    sig = [r]

    size_md = math.floor((k * a + 7) / 8)
    size_idx_tree = math.floor((h - h // d + 7) / 8)
    size_idx_leaf = math.floor((h // d + 7) / 8)
    digest = hash_msg(r, public_seed, public_root, m, size_md + size_idx_tree + size_idx_leaf)
    tmp_md = digest[:size_md]
    tmp_idx_tree = digest[size_md:(size_md + size_idx_tree)]
    tmp_idx_leaf = digest[(size_md + size_idx_tree):len(digest)]
    md_int = int.from_bytes(tmp_md, 'big') >> (len(tmp_md) * 8 - k * a)
    md = md_int.to_bytes(math.ceil(k * a / 8), 'big')
    idx_tree = int.from_bytes(tmp_idx_tree, 'big') >> (len(tmp_idx_tree) * 8 - (h - h // d))
    idx_leaf = int.from_bytes(tmp_idx_leaf, 'big') >> (len(tmp_idx_leaf) * 8 - (h // d))

    adrs.set_layer_address(0)
    adrs.set_tree_address(idx_tree)
    adrs.set_type(ADRS.FORS_TREE)
    adrs.set_key_pair_address(idx_leaf)

    sig_fors = fors_sign(md, secret_seed, public_seed, adrs.copy())
    sig += [sig_fors]
    pk_fors = fors_pk_from_sig(sig_fors, md, public_seed, adrs.copy())
    adrs.set_type(ADRS.TREE)
    sig_ht = ht_sign(pk_fors, secret_seed, public_seed, idx_tree, idx_leaf)
    sig += [sig_ht]

    return sig

# Input: Message M, signature SIG, public key PK
# Output: Boolean
def spx_verify(m, sig, public_key):
    adrs = ADRS()
    r = sig[0]
    sig_fors = sig[1]
    sig_ht = sig[2]
    public_seed = public_key[0]
    public_root = public_key[1]
    
    size_md = math.floor((k * a + 7) / 8)
    size_idx_tree = math.floor((h - h // d + 7) / 8)
    size_idx_leaf = math.floor((h // d + 7) / 8)
    digest = hash_msg(r, public_seed, public_root, m, size_md + size_idx_tree + size_idx_leaf)

    tmp_md = digest[:size_md]
    tmp_idx_tree = digest[size_md:(size_md + size_idx_tree)]
    tmp_idx_leaf = digest[(size_md + size_idx_tree):len(digest)]

    md_int = int.from_bytes(tmp_md, 'big') >> (len(tmp_md) * 8 - k * a)
    md = md_int.to_bytes(math.ceil(k * a / 8), 'big')
    idx_tree = int.from_bytes(tmp_idx_tree, 'big') >> (len(tmp_idx_tree) * 8 - (h - h // d))
    idx_leaf = int.from_bytes(tmp_idx_leaf, 'big') >> (len(tmp_idx_leaf) * 8 - (h // d))

    adrs.set_layer_address(0)
    adrs.set_tree_address(idx_tree)
    adrs.set_type(ADRS.FORS_TREE)
    adrs.set_key_pair_address(idx_leaf)

    pk_fors = fors_pk_from_sig(sig_fors, md, public_seed, adrs)

    adrs.set_type(ADRS.TREE)
    return ht_verify(pk_fors, sig_ht, public_seed, idx_tree, idx_leaf, public_root)

##### Implementação

De seguida, encontra-se um exemplo simples, para efeitos demonstrativos da aplicação do SPHINCS+.

In [8]:
# Geração chaves
sk, pk = spx_keygen()
mensagem = b'Tetsrferj e gwb l etrgbvrgj brgj rlk r vfgvgfvlfd fdjnbvkjdfbvjkfdbcjsdfbckjdbcvlds'
# Assinatura mensagem
assinatura = spx_sign(mensagem, sk)
# Verificação assinatura
print(spx_verify(mensagem, assinatura, pk))

True
